### requests 라이브러리

In [1]:
import requests

page = requests.get("https://dataquestio.github.io/web-scraping-pages/simple.html")
page

<Response [200]>

In [2]:
page.status_code

200

In [3]:
page.content

b'<!DOCTYPE html>\n<html>\n    <head>\n        <title>A simple example page</title>\n    </head>\n    <body>\n        <p>Here is some simple content for this page.</p>\n    </body>\n</html>'

### BeautifulSoup 라이브러리

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   A simple example page
  </title>
 </head>
 <body>
  <p>
   Here is some simple content for this page.
  </p>
 </body>
</html>


In [6]:
list(soup.children)

['html',
 '\n',
 <html>
 <head>
 <title>A simple example page</title>
 </head>
 <body>
 <p>Here is some simple content for this page.</p>
 </body>
 </html>]

In [7]:
[type(item) for item in list(soup.children)]

[bs4.element.Doctype, bs4.element.NavigableString, bs4.element.Tag]

In [8]:
html = list(soup.children)[2]
list(html.children)

['\n',
 <head>
 <title>A simple example page</title>
 </head>,
 '\n',
 <body>
 <p>Here is some simple content for this page.</p>
 </body>,
 '\n']

In [9]:
body = list(html.children)[3]
list(body.children)

['\n', <p>Here is some simple content for this page.</p>, '\n']

In [10]:
p = list(body.children)[1]
p.get_text()

'Here is some simple content for this page.'

### 태그명으로 태그 검색

In [11]:
soup = BeautifulSoup(page.content, 'html.parser')
soup.find_all('p')

[<p>Here is some simple content for this page.</p>]

In [12]:
soup.find_all('p')[0].get_text()

'Here is some simple content for this page.'

In [13]:
soup.find('p')

<p>Here is some simple content for this page.</p>

### class와 id로 태그 검색

In [14]:
page = requests.get("https://dataquestio.github.io/web-scraping-pages/ids_and_classes.html")
soup = BeautifulSoup(page.content, 'html.parser')
soup

<html>
<head>
<title>A simple example page</title>
</head>
<body>
<div>
<p class="inner-text first-item" id="first">
                First paragraph.
            </p>
<p class="inner-text">
                Second paragraph.
            </p>
</div>
<p class="outer-text first-item" id="second">
<b>
                First outer paragraph.
            </b>
</p>
<p class="outer-text">
<b>
                Second outer paragraph.
            </b>
</p>
</body>
</html>

In [15]:
soup.find_all('p', class_='outer-text')

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>,
 <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [16]:
soup.find_all(class_="outer-text")

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>,
 <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

### CSS 선택자

In [17]:
soup.select("div p")

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>,
 <p class="inner-text">
                 Second paragraph.
             </p>]

### 날씨 데이터 스크래핑

In [18]:
page = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Tonight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Tonight: Mostly clear, with a low around 50. Breezy, with a west northwest wind 16 to 26 mph, with gusts as high as 34 mph. " class="forecast-icon" src="DualImage.php?i=nwind_few&amp;j=nfew" title="Tonight: Mostly clear, with a low around 50. Breezy, with a west northwest wind 16 to 26 mph, with gusts as high as 34 mph. "/>
 </p>
 <p class="short-desc">
  Mostly Clear
  <br/>
  and Breezy
  <br/>
  then Mostly
  <br/>
  Clear
 </p>
 <p class="temp temp-low">
  Low: 50 °F
 </p>
</div>


In [19]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()
print(period)
print(short_desc)
print(temp)

Tonight
Mostly Clearand Breezythen MostlyClear
Low: 50 °F


In [20]:
img = tonight.find("img")
desc = img['title']
print(desc)

Tonight: Mostly clear, with a low around 50. Breezy, with a west northwest wind 16 to 26 mph, with gusts as high as 34 mph. 


In [21]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Tonight',
 'Monday',
 'MondayNight',
 'Tuesday',
 'TuesdayNight',
 'Wednesday',
 'WednesdayNight',
 'Thursday',
 'ThursdayNight']

In [22]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print(temps)
print(descs)

['Mostly Clearand Breezythen MostlyClear', 'Mostly Sunny', 'Mostly Clear', 'Sunny', 'Clear', 'Sunny', 'Clear', 'Sunny', 'Mostly Clear']
['Low: 50 °F', 'High: 64 °F', 'Low: 49 °F', 'High: 67 °F', 'Low: 50 °F', 'High: 78 °F', 'Low: 54 °F', 'High: 85 °F', 'Low: 55 °F']
['Tonight: Mostly clear, with a low around 50. Breezy, with a west northwest wind 16 to 26 mph, with gusts as high as 34 mph. ', 'Monday: Mostly sunny, with a high near 64. West wind 11 to 21 mph, with gusts as high as 26 mph. ', 'Monday Night: Mostly clear, with a low around 49. West northwest wind 11 to 17 mph, with gusts as high as 23 mph. ', 'Tuesday: Sunny, with a high near 67. Northwest wind 8 to 17 mph, with gusts as high as 22 mph. ', 'Tuesday Night: Clear, with a low around 50. West wind 13 to 18 mph becoming north 6 to 11 mph in the evening. Winds could gust as high as 23 mph. ', 'Wednesday: Sunny, with a high near 78.', 'Wednesday Night: Clear, with a low around 54.', 'Thursday: Sunny, with a high near 85.', 'Thu

In [23]:
import pandas as pd
weather = pd.DataFrame({
    "period": periods,
    "short_desc": short_descs,
    "temp": temps,
    "desc":descs
})
weather

,period,short_desc,temp,desc
0,Tonight,Mostly Clearand Breezythen MostlyClear,Low: 50 °F,"Tonight: Mostly clear, with a low around 50. B..."
1,Monday,Mostly Sunny,High: 64 °F,"Monday: Mostly sunny, with a high near 64. Wes..."
2,MondayNight,Mostly Clear,Low: 49 °F,"Monday Night: Mostly clear, with a low around ..."
3,Tuesday,Sunny,High: 67 °F,"Tuesday: Sunny, with a high near 67. Northwest..."
4,TuesdayNight,Clear,Low: 50 °F,"Tuesday Night: Clear, with a low around 50. We..."
5,Wednesday,Sunny,High: 78 °F,"Wednesday: Sunny, with a high near 78."
6,WednesdayNight,Clear,Low: 54 °F,"Wednesday Night: Clear, with a low around 54."
7,Thursday,Sunny,High: 85 °F,"Thursday: Sunny, with a high near 85."
8,ThursdayNight,Mostly Clear,Low: 55 °F,"Thursday Night: Mostly clear, with a low aroun..."


In [24]:
temp_nums = weather["temp"].str.extract(r'(?P<digit>\d+)', expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    50
1    64
2    49
3    67
4    50
5    78
6    54
7    85
8    55
Name: digit, dtype: object

In [25]:
weather["temp_num"].mean()

61.333333333333336

In [26]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

0     True
1    False
2     True
3    False
4     True
5    False
6     True
7    False
8     True
Name: temp, dtype: bool

In [27]:
weather[is_night]

,period,short_desc,temp,desc,temp_num,is_night
0,Tonight,Mostly Clearand Breezythen MostlyClear,Low: 50 °F,"Tonight: Mostly clear, with a low around 50. B...",50,True
2,MondayNight,Mostly Clear,Low: 49 °F,"Monday Night: Mostly clear, with a low around ...",49,True
4,TuesdayNight,Clear,Low: 50 °F,"Tuesday Night: Clear, with a low around 50. We...",50,True
6,WednesdayNight,Clear,Low: 54 °F,"Wednesday Night: Clear, with a low around 54.",54,True
8,ThursdayNight,Mostly Clear,Low: 55 °F,"Thursday Night: Mostly clear, with a low aroun...",55,True
